In [2]:
import pandas as pd

In [5]:
municipio = ['araputanga', 'figueiropolis','gloria_d_oeste', 
             'indiavai', 'jauru', 'lambari_doste',  
             'mirassol', 'porto', 'reserva',
             'rio_branco', 'salto', 'sao_jose_quatro_marcos'
            ]

In [3]:
import csv #importando csv

cidade_link = open('src/municipio_link.csv')

implementa = csv.DictReader(cidade_link)
#a leitura só funcionara com os parametros abaixo com o parametro acimas

for inplemente in implementa:
    #lendo arquivo python para rastrear o separador
    carregar_separador = open(inplemente['link'],encoding='ISO-8859-1')
    separa = csv.Sniffer().sniff(carregar_separador.read())
    separador = separa.delimiter
    print(separador)
    #---------------------------------------
    #dividindo municipio para colocar na variavel de nome    
    dividi_municipio = inplemente['municipio'].split(' ')# o codigo dividi o nome do municipio
    #----------------------------------------------
    
    # Carregando csv no pandas 
    servidores = pd.read_csv(inplemente['link']#chama o arquivo csv 
                             ,sep=separador #informa o separador do arquivo
                             ,encoding='ISO-8859-1', #informa a codificação do arquivo
                             error_bad_lines=False,
                             engine='python') #trata os erros nas linhas
    nome_variavel = 'servidores_{}'.format(dividi_municipio[0])     #alterando o nome da variavel
    globals()[nome_variavel] = servidores
    globals()[nome_variavel]['Municipio'] = inplemente['municipio']
    print(nome_variavel)
    print('-------------------------')



FileNotFoundError: [Errno 2] No such file or directory: 'src/municipio_link.csv'

In [4]:
'src/araputanga/ser'
'src/araputanga/'

'src/araputanga/'

In [ ]:
'src/figueiropolis/Portal Transp. Servidores_mes04.csv'